In [1]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy.stats import norm
from scipy import stats
import datetime
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression   
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import confusion_matrix,f1_score,classification_report
from sklearn.decomposition import PCA 
from sklearn import metrics
%matplotlib inline
pd.set_option('display.max_columns', 300)

In [2]:
# list of file names
files = os.listdir("../Tuple Technologies/Expression Data")
files[:10]

['NM.HA2.96.tiff',
 'KM.FE1.23.tiff',
 'KA.FE4.48.tiff',
 'UY.SU3.145.tiff',
 'KR.SU2.81.tiff',
 'YM.AN1.61.tiff',
 'KM.SA2.10.tiff',
 'KR.AN3.85.tiff',
 'KR.SU1.80.tiff',
 'UY.SA2.141.tiff']

In [3]:
# Reading each file and crating dataframe using each image as 256x256 pixel vector form
tiff_data = []
for f in files:
    img_vector = []
    split_name = f.split('.')
    img = Image.open("../Tuple Technologies/Expression Data/"+f)
    img_array = np.array(img)
    img_vector = list(img_array.ravel())
    img_vector.append(split_name[2])
    img_vector.append(split_name[1][-1:].strip())
    img_vector.append(split_name[1][:-1].strip())
    dfa = pd.DataFrame(img_vector)
    tiff_data.append(dfa.T)
df = pd.concat(tiff_data)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,...,65389,65390,65391,65392,65393,65394,65395,65396,65397,65398,65399,65400,65401,65402,65403,65404,65405,65406,65407,65408,65409,65410,65411,65412,65413,65414,65415,65416,65417,65418,65419,65420,65421,65422,65423,65424,65425,65426,65427,65428,65429,65430,65431,65432,65433,65434,65435,65436,65437,65438,65439,65440,65441,65442,65443,65444,65445,65446,65447,65448,65449,65450,65451,65452,65453,65454,65455,65456,65457,65458,65459,65460,65461,65462,65463,65464,65465,65466,65467,65468,65469,65470,65471,65472,65473,65474,65475,65476,65477,65478,65479,65480,65481,65482,65483,65484,65485,65486,65487,65488,65489,65490,65491,65492,65493,65494,65495,65496,65497,65498,65499,65500,65501,65502,65503,65504,65505,65506,65507,65508,65509,65510,65511,65512,65513,65514,65515,65516,65517,65518,65519,65520,65521,65522,65523,65524,65525,65526,65527,65528,65529,65530,65531,65532,65533,65534,65535,65536,65537,65538
0,153,151,144,156,154,144,149,155,151,153,154,149,150,148,145,153,146,146,154,138,148,159,155,149,156,162,162,161,163,162,166,165,161,160,154,160,164,168,166,175,160,157,169,162,160,165,159,163,168,160,158,157,167,164,159,168,149,160,165,163,166,161,169,166,164,164,168,167,171,172,175,172,170,172,178,164,180,165,175,178,172,180,176,171,183,177,182,166,182,185,171,175,172,175,173,179,173,184,173,180,167,172,174,176,173,170,181,175,178,175,181,174,176,180,181,180,181,179,176,176,172,195,174,173,183,180,186,177,177,175,174,179,179,179,189,178,172,175,171,175,183,169,179,182,182,174,186,166,181,170,...,137,138,135,130,127,142,132,138,145,146,145,135,160,162,176,187,179,185,185,185,188,191,183,183,187,197,190,187,187,185,192,185,180,180,178,180,175,186,176,183,163,182,192,193,181,178,176,172,184,165,169,172,172,137,130,77,32,11,9,9,7,7,1,29,32,36,37,43,72,125,141,170,175,192,190,178,182,172,172,180,180,168,174,171,175,184,179,169,174,183,181,175,174,166,165,173,169,161,154,157,163,161,160,166,160,159,146,157,142,153,158,147,151,133,129,130,118,115,116,113,106,97,102,103,98,98,95,101,98,108,97,105,104,113,104,105,116,111,106,104,106,114,114,112,100,107,102,96,2,HA
0,175,174,170,171,164,173,173,172,167,169,171,172,178,168,169,170,177,173,182,177,176,173,175,182,186,187,181,186,190,194,188,188,183,191,197,198,193,200,196,199,199,198,197,202,198,197,197,193,202,195,200,195,195,199,205,207,199,201,196,204,202,207,201,202,204,205,204,209,200,203,199,205,203,206,207,209,206,208,201,205,209,206,214,209,209,204,210,213,214,209,210,210,209,214,215,212,211,215,204,214,213,210,214,210,209,212,210,199,206,215,201,203,208,204,218,208,205,207,209,207,205,198,206,201,208,212,210,204,206,202,205,213,208,206,199,203,202,204,209,199,210,204,200,198,198,196,196,198,201,194,...,204,195,211,210,208,206,208,210,213,208,213,200,202,166,84,21,16,28,35,28,19,13,23,18,20,20,186,189,197,194,190,191,196,197,198,199,197,206,214,206,209,213,219,215,210,217,210,221,222,225,223,226,220,217,220,225,224,220,228,223,225,230,218,229,224,216,218,219,222,221,227,213,211,212,216,208,212,210,197,201,190,194,201,203,189,190,193,186,190,190,182,181,185,176,181,174,176,168,161,166,167,154,161,149,139,142,152,141,140,150,132,131,131,132,126,124,117,120,119,112,125,121,124,128,121,122,122,122,126,127,127,135,135,132,128,138,142,139,141,137,147,138,147,151,150,153,155,23,1,FE
0,85,82,95,85,77,79,87,82,88,85,96,101,95,100,108,108,99,105,109,106,109,109,111,110,113,118,121,128,127,147,127,144,137,139,141,151,145,145,142,153,140,143,139,153,153,146,157,143,155,165,150,163,148,150,155,145,142,145,140,146,157,158,1

In [4]:
# Renaming columns
df = df.rename(columns={65536: "ImageID", 65537: "NRE", 65538: "Expression"})
df.set_index('ImageID',inplace=True, drop=True)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,...,65388,65389,65390,65391,65392,65393,65394,65395,65396,65397,65398,65399,65400,65401,65402,65403,65404,65405,65406,65407,65408,65409,65410,65411,65412,65413,65414,65415,65416,65417,65418,65419,65420,65421,65422,65423,65424,65425,65426,65427,65428,65429,65430,65431,65432,65433,65434,65435,65436,65437,65438,65439,65440,65441,65442,65443,65444,65445,65446,65447,65448,65449,65450,65451,65452,65453,65454,65455,65456,65457,65458,65459,65460,65461,65462,65463,65464,65465,65466,65467,65468,65469,65470,65471,65472,65473,65474,65475,65476,65477,65478,65479,65480,65481,65482,65483,65484,65485,65486,65487,65488,65489,65490,65491,65492,65493,65494,65495,65496,65497,65498,65499,65500,65501,65502,65503,65504,65505,65506,65507,65508,65509,65510,65511,65512,65513,65514,65515,65516,65517,65518,65519,65520,65521,65522,65523,65524,65525,65526,65527,65528,65529,65530,65531,65532,65533,65534,65535,NRE,Expression
ImageID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
96,153,151,144,156,154,144,149,155,151,153,154,149,150,148,145,153,146,146,154,138,148,159,155,149,156,162,162,161,163,162,166,165,161,160,154,160,164,168,166,175,160,157,169,162,160,165,159,163,168,160,158,157,167,164,159,168,149,160,165,163,166,161,169,166,164,164,168,167,171,172,175,172,170,172,178,164,180,165,175,178,172,180,176,171,183,177,182,166,182,185,171,175,172,175,173,179,173,184,173,180,167,172,174,176,173,170,181,175,178,175,181,174,176,180,181,180,181,179,176,176,172,195,174,173,183,180,186,177,177,175,174,179,179,179,189,178,172,175,171,175,183,169,179,182,182,174,186,166,181,170,...,137,137,138,135,130,127,142,132,138,145,146,145,135,160,162,176,187,179,185,185,185,188,191,183,183,187,197,190,187,187,185,192,185,180,180,178,180,175,186,176,183,163,182,192,193,181,178,176,172,184,165,169,172,172,137,130,77,32,11,9,9,7,7,1,29,32,36,37,43,72,125,141,170,175,192,190,178,182,172,172,180,180,168,174,171,175,184,179,169,174,183,181,175,174,166,165,173,169,161,154,157,163,161,160,166,160,159,146,157,142,153,158,147,151,133,129,130,118,115,116,113,106,97,102,103,98,98,95,101,98,108,97,105,104,113,104,105,116,111,106,104,106,114,114,112,100,107,102,2,HA
23,175,174,170,171,164,173,173,172,167,169,171,172,178,168,169,170,177,173,182,177,176,173,175,182,186,187,181,186,190,194,188,188,183,191,197,198,193,200,196,199,199,198,197,202,198,197,197,193,202,195,200,195,195,199,205,207,199,201,196,204,202,207,201,202,204,205,204,209,200,203,199,205,203,206,207,209,206,208,201,205,209,206,214,209,209,204,210,213,214,209,210,210,209,214,215,212,211,215,204,214,213,210,214,210,209,212,210,199,206,215,201,203,208,204,218,208,205,207,209,207,205,198,206,201,208,212,210,204,206,202,205,213,208,206,199,203,202,204,209,199,210,204,200,198,198,196,196,198,201,194,...,210,204,195,211,210,208,206,208,210,213,208,213,200,202,166,84,21,16,28,35,28,19,13,23,18,20,20,186,189,197,194,190,191,196,197,198,199,197,206,214,206,209,213,219,215,210,217,210,221,222,225,223,226,220,217,220,225,224,220,228,223,225,230,218,229,224,216,218,219,222,221,227,213,211,212,216,208,212,210,197,201,190,194,201,203,189,190,193,186,190,190,182,181,185,176,181,174,176,168,161,166,167,154,161,149,139,142,152,141,140,150,132,131,131,132,126,124,117,120,119,112,125,121,124,128,121,122,122,122,126

In [5]:
df.Expression.value_counts()

FE    32
HA    31
SA    31
SU    30
AN    30
NE    30
DI    29
Name: Expression, dtype: int64

In [6]:
# distributing the dataset into two components X and Y 
X = df.iloc[:, 0:65536].values 
y = df.iloc[:, 65537].values 

In [7]:
# Splitting the X and Y into the Training set and Testing set 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0) 

Feature Scaling

In [8]:
sc = StandardScaler() 
X_train = sc.fit_transform(X_train) 
X_test = sc.transform(X_test) 

Applying PCA function

In [9]:
pca = PCA() 
X_train = pca.fit_transform(X_train) 
X_test = pca.transform(X_test) 
explained_variance = pca.explained_variance_ratio_

Fitting Logistic Regression To the training set

In [10]:
#Using multi_class='multinomial' for multi clss classification
classifier = LogisticRegression(multi_class='multinomial', solver='newton-cg') 
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

Predicting the test set result

In [11]:
y_pred = classifier.predict(X_test)

Making the confusion matrix

In [12]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[5, 1, 0, 0, 0, 0, 0],
       [0, 2, 2, 0, 0, 0, 0],
       [0, 1, 4, 0, 0, 0, 0],
       [0, 0, 0, 4, 2, 0, 0],
       [0, 0, 0, 0, 8, 0, 0],
       [0, 1, 0, 1, 0, 8, 0],
       [0, 0, 0, 0, 0, 0, 4]])

In [13]:
print("Train Accuracy(LR) ->", metrics.accuracy_score(y_train, classifier.predict(X_train)))
print("Test Accuracy(LR) ->", metrics.accuracy_score(y_test, classifier.predict(X_test)))
print("F1-Score(LR) ->", metrics.f1_score(y_test, classifier.predict(X_test),average="macro"))
print("Classification Report(LR) ->", metrics.classification_report(y_test, classifier.predict(X_test)))

Train Accuracy(LR) -> 1.0
Test Accuracy(LR) -> 0.813953488372093
F1-Score(LR) -> 0.7979797979797979
Classification Report(LR) ->               precision    recall  f1-score   support

          AN       1.00      0.83      0.91         6
          DI       0.40      0.50      0.44         4
          FE       0.67      0.80      0.73         5
          HA       0.80      0.67      0.73         6
          NE       0.80      1.00      0.89         8
          SA       1.00      0.80      0.89        10
          SU       1.00      1.00      1.00         4

    accuracy                           0.81        43
   macro avg       0.81      0.80      0.80        43
weighted avg       0.84      0.81      0.82        43



In [16]:
# Using SVM classifier 
from sklearn.svm import SVC 
svm_model = SVC(kernel = 'linear', C = 1).fit(X_train, y_train) 
svm_predictions = svm_model.predict(X_test) 
accuracy = svm_model.score(X_test, y_test) 
print("Accuracy(SVM)->",metrics.accuracy_score(y_train, svm_model.predict(X_train)))
print("Accuracy(SVM)->",accuracy)
print("F1-Score(SVM)->",metrics.f1_score(y_test, svm_model.predict(X_test),average="macro"))
print("Classification Report(SVM) ->", metrics.classification_report(y_test, svm_model.predict(X_test)))

Accuracy(SVM)-> 1.0
Accuracy(SVM)-> 0.7906976744186046
F1-Score(SVM)-> 0.7799847211611918
Classification Report(SVM) ->               precision    recall  f1-score   support

          AN       1.00      0.83      0.91         6
          DI       0.40      0.50      0.44         4
          FE       0.57      0.80      0.67         5
          HA       0.80      0.67      0.73         6
          NE       0.80      1.00      0.89         8
          SA       1.00      0.70      0.82        10
          SU       1.00      1.00      1.00         4

    accuracy                           0.79        43
   macro avg       0.80      0.79      0.78        43
weighted avg       0.83      0.79      0.80        43



Logistic Regression gives better accuracy on test data then SVM